<a href="https://colab.research.google.com/github/yohanesnuwara/reservoir-engineering/blob/master/Reservoir%20Simulation%20Ertekin/Unit%205%20Finite-Difference%20Approximation%20to%20Linear-Flow%20Equations/fd1d_linearflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Unit 5 Finite-Difference Approximations to Linear-Flow Problems**

## Explicit Formulation

$$p_{i}^{n+1} = p^i_n + (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot q_{lsc_i} + (\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i \cdot [T_{lx_{i+1/2}}^{n} p_{i+1}^{n}-(T_{lx_{i+1/2}}^{n} + T_{lx_{i-1/2}}^{n})p_{i}^{n}+T_{lx_{i-1/2}}p_{i-1}^{n}]$$

In the following codes, $(\frac{\alpha_c B_l \Delta t}{V_b \phi c_l})_i$ will be written as `factor`

Transmissibility of coupling cells, $T_{lx_{i\pm1/2}}^{n}$, written as `T_min_half` or `T_plus_half`

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
!git clone https://github.com/yohanesnuwara/reservoir-engineering

Cloning into 'reservoir-engineering'...
remote: Enumerating objects: 171, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 867 (delta 61), reused 0 (delta 0), pack-reused 696
Receiving objects: 100% (867/867), 4.95 MiB | 3.85 MiB/s, done.
Resolving deltas: 100% (359/359), done.


## Example 5.8 

![Example 5 8](https://user-images.githubusercontent.com/51282928/75949840-09e36380-5eda-11ea-824c-dd5c9ac6f6d5.PNG)

In [2]:
"Task. determine pressure distribution during first year of production with timestep 10 days"

# known
pi = 6000 # initial pressure, in psia
delta_x = 1000 # ft
delta_y = 1000
delta_z = 75
ngrid = 5
grid_loc = 4 # grid location where production well is located
B = 1 # phase FVF, assumed constant over pressure, rb/stb
c = 3.5E-06 # phase compressibility, psi^-1
k_x = 15 # perm in x direction, md
poro = 0.18
mu = 10 # phase viscosity, cp

delta_t = 10 # days
qsc = -150 # minus means production, stb/d

# conversion
k_x = 15 * 1E-03 # darcy to mD

# calculate factor
Vb = delta_x * delta_y * delta_z
alpha = 5.615 # volume conversion factor, is a constant
factor = (alpha * B * delta_t) / (Vb * poro * c)
factor

1.1883597883597885

Because of uniform gridblocks, the equation for `tr_plus_coupling` and `tr_min_coupling` become:

$$T_{lx_{i+1/2}}^{n} = T_{lx_{i-1/2}}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i+1/2}^{n} = (\beta_c \frac{A_x k_x}{\mu_l B_l \Delta x})_{i-1/2}^{n}$$

In [3]:
# calculate transmissibility of coupling cells
beta = 1.127 # transmissibility conversion factor, is a constant
Ax = delta_x * delta_z
T_plus_half = beta * ((Ax * k_x) / (mu * B * delta_x))
T_min_half = T_plus_half
T_min_half, T_plus_half

(0.1267875, 0.1267875)

There are 5 grids (`grid_1`, `grid_2`, `grid_3`, `grid_4`, `grid_5`), so the transmissibility coefficients will be: $$T_{lx_{i+1/2}}^{n}=(T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n}, T_{lx_{5+1/2}}^{n})$$

and

$$T_{lx_{i-1/2}}^{n}=(T_{lx_{1/2}}^{n}, T_{lx_{1+1/2}}^{n}, T_{lx_{2+1/2}}^{n}, T_{lx_{3+1/2}}^{n}, T_{lx_{4+1/2}}^{n})$$

The values of $T_{lx_{5+1/2}}^{n}=0$ and $T_{lx_{1/2}}^{n}=0$ (the edge of gridblocks), so the values of each are:

$$T_{lx_{i+1/2}}^{n}=(0.127, 0.127, 0.127, 0.127, 0)$$

and

$$T_{lx_{i-1/2}}^{n}=(0, 0.127, 0.127, 0.127, 0.127)$$




In [4]:
q = np.full(ngrid-1, T_min_half)
Ti_plus_halves = np.append(q, [0])
print(Ti_plus_halves)
p = np.full(ngrid-1, T_plus_half)
Ti_min_halves = np.append([0], p)
print(Ti_min_halves)
print("At grid 1, the coupling transmissibility coeffs are:", Ti_min_halves[0], "for T_min_half and:", Ti_plus_halves[0], "for T_plus_half.")
print("At grid 3, the coupling transmissibility coeffs are:", Ti_min_halves[2], "for T_min_half and:", Ti_plus_halves[2], "for T_plus_half.")
print("At grid 5, the coupling transmissibility coeffs are:", Ti_min_halves[4], "for T_min_half and:", Ti_plus_halves[4], "for T_plus_half.")

[0.1267875 0.1267875 0.1267875 0.1267875 0.       ]
[0.        0.1267875 0.1267875 0.1267875 0.1267875]
At grid 1, the coupling transmissibility coeffs are: 0.0 for T_min_half and: 0.1267875 for T_plus_half.
At grid 3, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.1267875 for T_plus_half.
At grid 5, the coupling transmissibility coeffs are: 0.1267875 for T_min_half and: 0.0 for T_plus_half.


Requires array for $q_{sc}$

In [0]:
qsc = [0, 0, 0, -150, 0] # production well in grid 4

Calculate $p_{i}^{n+1}$ for each grid in each time

In [11]:
# pi = [6000, 6000, 6000, 6000, 6000]
pi = np.full(ngrid, 6000) # array of pressure in each grid [6000, 6000, 6000, 6000, 6000]
time = np.arange(10, 370, delta_t)

pi_arr = []
min_arr = []
plus_arr = []

for j in range(len(time)):
  pnew_arr = []

  minus = pi[0]
  plus = pi[-1]
  minus_arr = []
  plusus_arr = []
  for i, obj in enumerate(pi):
    if i > 0:
      minus = pi[i-1]
    if i < (len(pi) - 1):
      plus = pi[i+1]

    pnew = pi[i] + (factor * ((Ti_plus_halves[i] * plus) - ((Ti_plus_halves[i] + Ti_min_halves[i]) * pi[i]) + (Ti_min_halves[i] * minus))) + (factor * qsc[i])
    pnew_arr.append(float(pnew))

    minus_arr.append(float(minus))
    plusus_arr.append(float(plus))

  pi = pnew_arr

  min_arr.append(minus_arr)
  plus_arr.append(plusus_arr)
  pi_arr.append(pi)

df = pd.DataFrame.from_records(pi_arr)
df = pd.DataFrame(pd.np.column_stack([time, df]), columns=['time', 'grid 1', 'grid 2', 'grid 3', 'grid 4', 'grid 5'])
df

,time,grid 1,grid 2,grid 3,grid 4,grid 5
0,10.0,6000.000000,6000.000000,6000.000000,5821.746032,6000.000000
1,20.0,6000.000000,6000.000000,5973.142623,5697.206817,5973.142623
2,30.0,6000.000000,5995.953421,5935.614184,5602.102885,5931.567605
3,40.0,5999.390305,5987.471853,5894.455577,5523.738961,5881.927430
4,50.0,5997.594562,5975.252912,5852.614698,5455.308514,5827.959472
5,60.0,5994.228364,5960.141312,5811.230704,5393.063347,5771.812463
6,70.0,5989.092497,5942.840943,5770.662014,5334.880120,5714.746649
7,80.0,5982.123813,5923.867570,5730.945175,5279.519220,5657.512476
8,90.0,5973.346394,5903.577533,5691.996659,5226.233153,5600.560547
9,100.0,5962.834378,5882.210835,5653.699174,5174.554980,5544.160950
